In [6]:
import pandas as pd

df = pd.DataFrame({
    'A': [4, 6, 6, 3],
    'B': [2, 3, 4, 8],
    'C': [2, 4, 4, 8],
    'D': [1, 3, 8, 4]
})

In [18]:
df

,A,B,C,D
0,4,2,2,1
1,6,3,4,3
2,6,4,4,8
3,3,8,8,4


In [19]:
df['A']

0    4
1    6
2    6
3    3
Name: A, dtype: int64

In [11]:
df[['A', 'C']]

,A,C
0,4,2
1,6,4
2,6,4
3,3,8


In [34]:
# df['B:D']  # - doesn't work
# df['B':'D']  # - doesn't work
# df.ix[:, 'B':'D']  # - deprecated: use df.loc / df.iloc instead
df.loc[:, 'B':'D']

,B,C,D
0,2,2,1
1,3,4,3
2,4,4,8
3,8,8,4


In [24]:
df.loc[1:2, 'B':'D']

,B,C,D
1,3,4,3
2,4,4,8


In [37]:
df.loc[[0,2],['B','D']]

,B,D
0,2,1
2,4,8


## Missing data

In [42]:
s1 = pd.Series([1,2,3,4], index=list("ABCD"))
s2 = pd.Series([5,6,7,8], index=list("CDEF"))

df = pd.DataFrame({'a': s1, 'b': s2})

In [58]:
df  # we have NaN's where the missing values are:

,a,b
A,1.0,NaN
B,2.0,NaN
C,3.0,5.0
D,4.0,6.0
E,NaN,7.0
F,NaN,8.0


In [47]:
df2 = pd.DataFrame([s1, s2])
df2

,A,B,C,D,E,F
0,1.0,2.0,3.0,4.0,NaN,NaN
1,NaN,NaN,5.0,6.0,7.0,8.0


In [52]:
df3 = df.dropna()  # drop missing values
df3

,a,b
C,3.0,5.0
D,4.0,6.0


In [55]:
df4 = df2.fillna(0)  # fill in missing values with zeros
df4

,A,B,C,D,E,F
0,1.0,2.0,3.0,4.0,0.0,0.0
1,0.0,0.0,5.0,6.0,7.0,8.0


In [57]:
df5 = df.reindex(index=['D','E','F','G'], columns=['b','c'])  # new labels are "out of bounds" of df
df5  # again, missing values are filled in with NaN's

,b,c
D,6.0,NaN
E,7.0,NaN
F,8.0,NaN
G,NaN,NaN


## Hierarchical Index

```
                      |  X   Y   Z   R
Location   Day   Type |  
----------------------|-----------------
Paris      Mon   A    |  ..  ..  ..  ..
Paris      Tue   B    |  ..  ..  ..  ..
...        ...   ..   |  ..  ..  ..  ..
London     Mon   A    |  ..  ..  ..  ..
...

```

Here we have the 2D array of data, but along with the hierarchical index (Location, Day, Type), it represents the multi-dimentional data.

In [96]:
multi_index = pd.MultiIndex.from_arrays(
    [
        ['Py', 'Py', 'Rb', 'Rb'],
        ['Speed', 'Lines', 'Speed', 'Lines'],
    ],
    names=['Lang', 'Cat']
)
s1 = pd.Series([8, 2, 3, 4], index=multi_index)
s1

Lang  Cat  
Py    Speed    8
      Lines    2
Rb    Speed    3
      Lines    4
dtype: int64

In [101]:
df = pd.DataFrame({'test1': [8, 2, 3, 4], 'test2': [5, 1, 2, 2]}, index=multi_index)
df

test1  test2
Lang Cat                
Py   Speed      8      5
     Lines      2      1
Rb   Speed      3      2
     Lines      4      2

In [102]:
df['test1']  # Get 'test1' data

Lang  Cat  
Py    Speed    8
      Lines    2
Rb    Speed    3
      Lines    4
Name: test1, dtype: int64

If we want to get lines, corresponding to the inner index (for example, `Speed`), we need to use `xs` method (cross-slice):

In [103]:
df.xs('Speed', level='Cat')

,test1,test2
Lang,,
Py,8,5
Rb,3,2


In [104]:
df.xs('Speed', level=1)  # same as above, specify `level` with integer

,test1,test2
Lang,,
Py,8,5
Rb,3,2


In [110]:
df.xs('test2', axis=1)  # we can slice by columns using `axis=1`

Lang  Cat  
Py    Speed    5
      Lines    1
Rb    Speed    2
      Lines    2
Name: test2, dtype: int64

## Merging

In [78]:
df1 = pd.DataFrame({
    'name': ['John', 'Bill', 'Jane'],
    'age': [31, 22, 44],
})
df2 = pd.DataFrame({
    'name': ['John', 'Bill', 'Jax'],
    'weight': [80, 90, 66],
})

result = pd.merge(df1, df2, left_on='name', right_on='name')
result

,age,name,weight
0,31,John,80
1,22,Bill,90


In [81]:
result = pd.merge(df1, df2, how='left')
result

,age,name,weight
0,31,John,80.0
1,22,Bill,90.0
2,44,Jane,NaN


### Join

In [83]:
df1 = pd.DataFrame({
    'age': [31, 22, 44],
}, index=['John', 'Bill', 'Jane'])
df2 = pd.DataFrame({
    'weight': [80, 90, 66],
}, index=['John', 'Bill', 'Jane'])
df3 = pd.DataFrame({
    'score': [180, 190, 166],
}, index=['John', 'Bill', 'Jane'])

result = df1.join([df2, df3])  # `join` can merge more than one dataset at once,
                               # but columns should be different in all datasets
result

,age,weight,score
John,31,80,180
Bill,22,90,190
Jane,44,66,166


### Concat - "stack" dataframe rows together

In [90]:
df1 = pd.DataFrame({
    'age': [31, 22, 44],
    'height': [155, 167, 177]
}, index=['John', 'Bill', 'Jane'])
df2 = pd.DataFrame({
    'weight': [80, 90, 66],
}, index=['John', 'Bill', 'Jane'])
df3 = pd.DataFrame({
    'score': [180, 190, 166],
}, index=['John', 'Bill', 'Jane'])

result = pd.concat([df1, df2, df3], keys=['data1', 'data2', 'data3']) # we can use `keys` to create MultiIndex
result

age  height  score  weight
data1 John  31.0   155.0    NaN     NaN
      Bill  22.0   167.0    NaN     NaN
      Jane  44.0   177.0    NaN     NaN
data2 John   NaN     NaN    NaN    80.0
      Bill   NaN     NaN    NaN    90.0
      Jane   NaN     NaN    NaN    66.0
data3 John   NaN     NaN  180.0     NaN
      Bill   NaN     NaN  190.0     NaN
      Jane   NaN     NaN  166.0     NaN

But we can also use `concat` to join by columns using `axis=1` parameter:

In [91]:
result = pd.concat([df1, df2, df3], axis=1, keys=['d1', 'd2', 'd3']) # we can use `axis=1` to concat by columns
result

d1            d2    d3
     age height weight score
John  31    155     80   180
Bill  22    167     90   190
Jane  44    177     66   166

In [92]:
result['d1']

,age,height
John,31,155
Bill,22,167
Jane,44,177


## Stack / unstack

In [112]:
df1 = pd.DataFrame({
    'name': ['John', 'Bill', 'Jane'],
    'age': [31, 22, 44],
})

df1.stack()  ## Stack turns dataframe into a series
             ## It increases the 'height' of the dataset

0  age       31
   name    John
1  age       22
   name    Bill
2  age       44
   name    Jane
dtype: object

An example of mutli-dimentional dataset (with MultiIndex):

In [117]:
multi_index = pd.MultiIndex.from_arrays(
    [
        ['Py', 'Py', 'Rb', 'Rb'],
        ['Speed', 'Lines', 'Speed', 'Lines'],
    ],
    names=['Lang', 'Cat']
)
df = pd.DataFrame({'test1': [8, 2, 3, 4], 'test2': [5, 1, 2, 2]}, index=multi_index)
df

test1  test2
Lang Cat                
Py   Speed      8      5
     Lines      2      1
Rb   Speed      3      2
     Lines      4      2

In [118]:
df.stack()

Lang  Cat         
Py    Speed  test1    8
             test2    5
      Lines  test1    2
             test2    1
Rb    Speed  test1    3
             test2    2
      Lines  test1    4
             test2    2
dtype: int64

In [119]:
df.unstack()  # Unstack increases "width" of the dataset:

test1       test2      
Cat  Lines Speed Lines Speed
Lang                        
Py       2     8     1     5
Rb       4     3     2     2

In [120]:
df.unstack().unstack()  # And we finish by turning it into series too

       Cat    Lang
test1  Lines  Py      2
              Rb      4
       Speed  Py      8
              Rb      3
test2  Lines  Py      1
              Rb      2
       Speed  Py      5
              Rb      2
dtype: int64

In [123]:
df.unstack().unstack().unstack()  # If we continue, we get the dataframe again, with inner index moved to columns

Lang         Py  Rb
      Cat          
test1 Lines   2   4
      Speed   8   3
test2 Lines   1   2
      Speed   5   2

In [126]:
print(df1)
print(df1.stack().unstack(0))  # Exchange rows and columns
print(df1.unstack().unstack())  # Same, exchange rows and columns

   age  name
0   31  John
1   22  Bill
2   44  Jane
         0     1     2
age     31    22    44
name  John  Bill  Jane
         0     1     2
age     31    22    44
name  John  Bill  Jane


## References

[A Visual Guide to Pandas](https://www.youtube.com/watch?v=9d5-Ti6onew)